In [1]:
from pathlib import Path
from ase import io as ase_io

from aiida import orm, engine, load_profile
from aiida.common.exceptions import NotExistent

load_profile()

Profile<uuid='636ce0ecaed848c18631033f4daed624' name='test'>

# Setup for running Quantum ESPRESSO

The cells below will setup the `localhost` computer and `pw.x` code on this computer.

In [2]:
try:
    localhost = orm.load_computer('localhost')
except NotExistent:
    localhost = orm.Computer(
        label='localhost',
        hostname='localhost',
        transport_type='core.local',
        scheduler_type='core.direct',
        workdir=Path('workdir').absolute().as_posix()
    ).store().configure()

try:
    pw_code = orm.load_code('pw@localhost')
except NotExistent:
    pw_code = orm.InstalledCode(
        label='pw',
        computer=localhost,
        filepath_executable='pw.x',
        default_calc_job_plugin='aiida_qe_basic.pw',
        prepend_text='export OMP_NUM_THREADS=1'
    ).store()

With the cell below you can set up a basic `pw.x` run via the plugin defined in the `aiida_qe_basic` package.

In [3]:
from aiida_qe_basic.pw import PwCalculation

builder = PwCalculation.get_builder()

builder.code = pw_code
builder.structure = orm.StructureData(ase=ase_io.read('files/Al.cif'))
builder.pseudopotentials = orm.Dict({"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"})
builder.parameters = orm.Dict(
    {
        'CONTROL': {
            'calculation': 'scf',
            'pseudo_dir': Path('files').absolute().as_posix(),
        },
        'SYSTEM': {
            'occupations': 'smearing',
            'smearing': 'cold',
            'degauss': 0.02
        }
    }
)
builder.metadata.options.resources = {
    'num_machines': 1,
    'num_mpiprocs_per_machine': 1
}

In [4]:
results = engine.run(builder)

In [5]:
results

{'structure': <StructureData: uuid: 6ba15262-c338-4ecd-a721-3c32e8ab9967 (pk: 16)>,
 'properties': <Dict: uuid: b268c1e1-61d7-45ef-ab17-438ee5f6067c (pk: 17)>,
 'remote_folder': <RemoteData: uuid: b3aa241b-a336-4396-8cb4-26162dd005ce (pk: 14)>,
 'retrieved': <FolderData: uuid: 05b371b1-0b0a-4bec-806a-c89d75db7ec1 (pk: 15)>}

In [6]:
results['properties'].get_dict()

{'energy': -536.90337599314, 'volume': 16.48873596666}